In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [3]:

from pandas.io import sql as psql

In [4]:
from lib.connected_db import PgsqlExecutor

In [5]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [6]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

# Long investment - consistent growth up

In [7]:
def get_long_stock_data(year):
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    order by adjusted_dcf_to_price desc
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data

def get_short_stock_data(year):
    """
    You don't want to short pennies stock, you only care when stock > 10
    """
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    
    and price > 10
    order by adjusted_dcf_to_price
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data
    


In [8]:
REPORT = """
**********- Score: {score} - {symbol} - {exchange}: {start_price} on {start_date} - *********
End date: {end_date} - price: {end_price} - min: {min_price} - max: {max_price}
Profit: {perf_profit} - max potential: {perf_potential} 
Average growth rate: {avg_growth}, consistent: {is_consistent}, project growth: {adj_growth}
DCF: {dcf}, market_dcf: {market_dcf}, adjusted_dcf: {adj_dcf}, dcf_to_price : {dcf_to_price}

"""
import numpy

def report_all(year):
    profits = []
    potentials = []
    data = get_long_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.year_end_price - row.price) * 100 / row.price
            ,1)
        potential = round(
            (row.year_max_price - row.price) * 100 / row.price
            ,1)
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% ***".format(
        numpy.mean(profits), numpy.mean(potentials)
    ))
    return numpy.mean(profits), numpy.mean(potentials)

def report_short_all(year):
    profits = []
    potentials = []
    data = get_short_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.price - row.year_end_price) * 100 / row.price
            ,1)
        potential = round(
            (row.price - row.year_min_price) * 100 / row.price
            ,1)
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% ***".format(
        numpy.mean(profits), numpy.mean(potentials)
    ))
    return numpy.mean(profits), numpy.mean(potentials)


# Including Pennies Stock - 2019

In [9]:
report_all(2019)


**********- Score: -24.700000762939453 - TWMC - NasdaqCM: 2.01 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 6.92 - min: 1.92 - max: 7.745
Profit: 244.3 - max potential: 285.3 
Average growth rate: -0.419, consistent: 0, project growth: -0.805
DCF: 55443.3, market_dcf: 15135.4, adjusted_dcf: 184538.2, dcf_to_price : 9180904.98



**********- Score: 31.799999237060547 - OPGN - NasdaqCM: 1.13 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 2.01 - min: 1.13 - max: 4.04
Profit: 77.9 - max potential: 257.5 
Average growth rate: -0.063, consistent: 0, project growth: -0.125
DCF: 1729.1, market_dcf: 2006.5, adjusted_dcf: 2294.6, dcf_to_price : 202961.95



**********- Score: 19.600000381469727 - GURE - NasdaqGS: 2.5505 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 5.81 - min: 2.5505 - max: 5.89
Profit: 127.8 - max potential: 130.9 
Average growth rate: -0.63, consistent: 0, project growth: -0.893
DCF: 1765.8, m

(8.446666666666669, 67.92)

# 2018

In [28]:
report_all(2018)


**********- Score: -27.700000762939453 - BLDR - NasdaqGS: 10.91 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 25.41 - min: 10.91 - max: 25.92
Profit: 132.9 - max potential: 137.6 
Average growth rate: 177.359, consistent: 0, project growth: 105.969
DCF: 198259638096.2, market_dcf: 30.2, adjusted_dcf: 15438147991.7, dcf_to_price : 141504564443.54



**********- Score: -0.8999999761581421 - ENTA - NasdaqGS: 70.83 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 61.78 - min: 57.95 - max: 105.66
Profit: -12.8 - max potential: 49.2 
Average growth rate: 165.44, consistent: 0, project growth: 99.311
DCF: 58171285728.5, market_dcf: 66.4, adjusted_dcf: 4733102156.9, dcf_to_price : 6682340937.55



**********- Score: -14.699999809265137 - RIOT - NasdaqCM: 1.51 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 1.12 - min: 1.12 - max: 6.02
Profit: -25.8 - max potential: 298.7 
Average growth rate: 16.843, consistent: 0,

(3.7833333333333345, 61.776666666666664)

In [29]:
initial_investment = 1000
potential_gain = initial_investment
yearly_cumulated_gain = initial_investment
potential_yearly_gains = []
profit_yearly_gains = []

for year in range(2005, 2010):
    print (year)
    profit_per, potential_per = report_all(year)
    profit_yearly_gains.append(profit_per)
    potential_yearly_gains.append(potential_per)
    
    yearly_cumulated_gain = yearly_cumulated_gain * (1 + profit_per/100)
    potential_gain = potential_gain *(1 + potential_per/100)



2005

**********- Score: -3.0999999046325684 - BBBY - NasdaqGS: 36.15 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 38.1 - min: 31.42 - max: 41.24
Profit: 5.4 - max potential: 14.1 
Average growth rate: 300.689, consistent: 0, project growth: 180.471
DCF: 2829012326041.2, market_dcf: 37.9, adjusted_dcf: 223239551250.2, dcf_to_price : 617536794506.36



**********- Score: -3.5 - POOL - NasdaqGS: 37.22 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 39.17 - min: 36.02 - max: 49.79
Profit: 5.2 - max potential: 33.8 
Average growth rate: 178.444, consistent: 0, project growth: 107.37
DCF: 32160737471.4, market_dcf: 42.2, adjusted_dcf: 2641085901.9, dcf_to_price : 7095878196.35



**********- Score: -4.800000190734863 - IEC - NasdaqGM: 0.62 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 1.5 - min: 0.51 - max: 1.75
Profit: 141.9 - max potential: 182.3 
Average growth rate: 47.821, consistent: 0, project growth: 



**********- Score: -8.600000381469727 - LDL - NYSE: 10.52 on 2007-12-31 00:00:00 - *********
End date: 2008-12-30 00:00:00 - price: 5.15 - min: 3.31 - max: 16.29
Profit: -51.0 - max potential: 54.8 
Average growth rate: 9.655, consistent: 0, project growth: 5.39
DCF: 73351.5, market_dcf: 13.2, adjusted_dcf: 6724.3, dcf_to_price : 63819.2



**********- Score: -47.20000076293945 - LCUT - NasdaqGS: 12.98 on 2007-12-31 00:00:00 - *********
End date: 2008-12-30 00:00:00 - price: 3.0 - min: 3.0 - max: 13.07
Profit: -76.9 - max potential: 0.7 
Average growth rate: 10.21, consistent: 0, project growth: 0.575
DCF: 127622800.9, market_dcf: 1128.1, adjusted_dcf: 6961.6, dcf_to_price : 53533.28


*** Average Profit: -48.606666666666676% , Potential: 50.39333333333334% ***
2008

**********- Score: 29.100000381469727 - MATX - NYSE: 25.06 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 34.23 - min: 16.21 - max: 35.37
Profit: 36.6 - max potential: 41.1 
Average growth rate

In [30]:
print("""
Initial Investment: ${}
Average yearly gain : {},
Average potential gain: {}
Cumulated potential capital: ${}
Cumulated capital yearly end: ${}

""".format(
    initial_investment,
    round(numpy.mean(profit_yearly_gains)),
    round(numpy.mean(potential_yearly_gains)),
    '{:0,.2f}'.format(yearly_cumulated_gain),
    '{:0,.2f}'.format(potential_gain),
    
))


Initial Investment: $1000
Average yearly gain : 3.0,
Average potential gain: 61.0
Cumulated potential capital: $822.19
Cumulated capital yearly end: $10,717.93




# Shorting - current year 2019

In [10]:
report_short_all(2019)


**********- Score: -45.599998474121094 - DCTH - NasdaqCM: 21.25 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 8.9 - min: 7.31 - max: 21.25
Profit: 58.1 - max potential: 65.6 
Average growth rate: -0.087, consistent: 0, project growth: -0.175
DCF: -33564.8, market_dcf: -34731.2, adjusted_dcf: -49036.5, dcf_to_price : -230860.0



**********- Score: -5.599999904632568 - ORI - NYSE: 22.37 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 16.49 - min: 12.33 - max: 23.49
Profit: 26.3 - max potential: 44.9 
Average growth rate: -1.113, consistent: 0, project growth: -2.182
DCF: -5986.9, market_dcf: 51.1, adjusted_dcf: -47001.6, dcf_to_price : -210209.97



**********- Score: -21.100000381469727 - SAFE - NYSE: 40.3 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 66.0 - min: 39.42 - max: 66.01
Profit: -63.8 - max potential: 2.2 
Average growth rate: -1.213, consistent: 0, project growth: -1.689
DCF: -23338.8, market

(17.15, 42.20666666666666)

In [14]:
report_short_all(2018)


**********- Score: 12.699999809265137 - AMBC - NYSE: 17.24 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 21.57 - min: 14.54 - max: 21.74
Profit: -25.1 - max potential: 15.7 
Average growth rate: 13.612, consistent: 0, project growth: 8.082
DCF: -25163687770.0, market_dcf: -59489.5, adjusted_dcf: -2333937225.0, dcf_to_price : -13537919038.52



**********- Score: -34.70000076293945 - DVN - NYSE: 22.54 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 25.97 - min: 19.8 - max: 35.16
Profit: -15.2 - max potential: 12.2 
Average growth rate: 21.395, consistent: 0, project growth: 12.965
DCF: -5069849435.5, market_dcf: -1422.9, adjusted_dcf: -478100251.1, dcf_to_price : -2121119226.44



**********- Score: 21.100000381469727 - HALL - NasdaqGM: 10.69 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 17.57 - min: 9.57 - max: 19.88
Profit: -64.4 - max potential: 10.5 
Average growth rate: 11.905, consistent: 0, project

(5.3133333333333335, 29.376666666666665)

In [16]:
!ipython nbconvert --to html Reporting_Rank3_2020_06_16_adjust_dcf_to_market.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook Reporting_Rank3_2020_06_16_adjust_dcf_to_market.ipynb to html
[NbConvertApp] Writing 462717 bytes to Reporting_Rank3_2020_06_16_adjust_dcf_to_market.html
